In [1]:
import requests
from bs4 import BeautifulSoup


In [92]:
#insert the url of the mixing forum from Cambridge MT that you want to scrape
#find the forum here: https://discussion.cambridge-mt.com/forumdisplay.php?fid=184 
url = "https://discussion.cambridge-mt.com/forumdisplay.php?fid=6"
#get the html content of the page
html = requests.get(url).content
print(html.decode("utf-8"))


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><!-- start: forumdisplay -->
<html xml:lang="en" lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Discussion Zone - Rock, Punk, Metal </title>
<!-- start: headerinclude -->
<link rel="alternate" type="application/rss+xml" title="Latest Threads (RSS 2.0)" href="https://discussion.cambridge-mt.com/syndication.php" />
<link rel="alternate" type="application/atom+xml" title="Latest Threads (Atom 1.0)" href="https://discussion.cambridge-mt.com/syndication.php?type=atom1.0" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="Content-Script-Type" content="text/javascript" />
<script type="text/javascript" src="https://discussion.cambridge-mt.com/jscripts/jquery.js?ver=1821"></script>
<script type="text/javascript" src="https://discussion.cambridge-mt.com/jscripts/jquery.plugins.min.js?ver=1821"></script>
<script type="text

In [93]:
# Parse the HTML content
soup = BeautifulSoup(html, 'html.parser')

# Find all <strong> tags containing song names and usernames
strong_tags = soup.find_all('strong')
aaa_tags = soup.find_all('a')
print(aaa_tags)
#if the aaa_tags['href'] is a url, then add to the list of urls
url = []
#create a dictionary to store the song names and the urls to the forums pertaining to the song
song_dict = {}
# Extract links to forums with song names and usernames
for strong_tag in strong_tags:
    a_tag = strong_tag.find('a')
    #print(a_tag)
    if a_tag:
        forum_link = "https://discussion.cambridge-mt.com/" + a_tag['href']
        song_name = a_tag.get_text()
        song_dict[song_name] = {}


        song_dict[song_name]['forum_link'] = forum_link
        # print("Song Name:", song_name)
        # print("Link:", forum_link)
    # if a_tag in aaa_tags:
    #     #find the index of the a_tag in aaa_tags
    #     index = aaa_tags.index(a_tag)

    #     username = aaa_tags[index + 2].get_text()
    #     user_link = aaa_tags[index + 2]['href']
    #     print("Username:", username)
    #     print("User Link:", user_link)
print(song_dict)
print(len(song_dict))


[<a id="top" name="top"></a>, <a href="https://discussion.cambridge-mt.com/index.php"><img alt="Discussion Zone" src="https://discussion.cambridge-mt.com/images/CMTlogo.gif" title="Discussion Zone"/></a>, <a class="portal" href="https://discussion.cambridge-mt.com/portal.php">Portal</a>, <a class="search" href="https://discussion.cambridge-mt.com/search.php">Search</a>, <a class="memberlist" href="https://discussion.cambridge-mt.com/memberlist.php">Member List</a>, <a class="help" href="https://discussion.cambridge-mt.com/misc.php?action=help">Help</a>, <a class="login" href="https://discussion.cambridge-mt.com/member.php?action=login" onclick="$('#quick_login').modal({ fadeDuration: 250, keepelement: true, zIndex: (typeof modal_zindex !== 'undefined' ? modal_zindex : 9999) }); return false;">Login</a>, <a class="register" href="https://discussion.cambridge-mt.com/member.php?action=register">Register</a>, <a class="lost_password" href="https://discussion.cambridge-mt.com/member.php?act

In [94]:
#now we will extract the link to the threads within a song forum along with the number of views, ratings (between 0-5) and the author of the thread
for key, value in song_dict.items():
    print(key)
    song_forum_url = value['forum_link']
    
    song_forum_html = requests.get(song_forum_url).content
    song_forum_soup = BeautifulSoup(song_forum_html, 'html.parser')
    # Extracting the number of pages
    pages_element = song_forum_soup.find('span', class_='pages')
    if pages_element:
        pages_text = pages_element.get_text(strip=True)
        # Extracting the number of pages from the text
        pages_number = int(pages_text.split('(')[1].split(')')[0])
        print("Number of pages:", pages_number)
    else:
        print("No page information found.")
    threads = []
    #parse through all the pages and add it to html
    for i in range(1, pages_number + 1):

        if i == 1:
            song_forum_url = value['forum_link']
        else:
            song_forum_url = value['forum_link'] + "&page=" + str(i)
        song_forum_html = requests.get(song_forum_url).content
        song_forum_soup = BeautifulSoup(song_forum_html, 'html.parser')
        mixes = song_forum_soup.find_all('tr', class_='inline_row')

        # Extracting data
      
        for row in mixes:
            try:
                thread_link = row.find('span', class_='subject_new').a['href']
            except:
                thread_link = "none"
            try:
                #print(thread_link)
                thread_title = row.find('span', class_='subject_new').text
            except:
                thread_title = "none"
                #print(thread_title)
            try:
                thread_author_link = row.find('div', class_='author').a['href']
            except:
                thread_author_link = "none"
            try:
                thread_author = row.find('div', class_='author').a.text
            except:
                thread_author = "none"
            try:
                #print(thread_author)
                thread_rating = row.find('ul', class_='star_rating').find('li').text.strip()
            except:
                thread_rating = "none"

            #print(thread_rating)
            try:
                thread_views = row.find_all('td', class_='trow1')[4].text.strip()
            except:
                thread_views = row.find_all('td', class_='trow2')[4].text.strip()
            #print(thread_views)
            
            
            threads.append({
                'Thread Link': thread_link,
                'Thread Title': thread_title,
                'Thread Author': thread_author,
                'Thread Author Link': thread_author_link,
                'Thread Rating': thread_rating,
                'Thread Views': thread_views
            })
    print(len(threads))
    song_dict[key]['threads'] = threads
    

print(song_dict)
    

Ale Lak: 'Nosso Mundo Deixou De Existir'
Number of pages: 4
70
All Hands Lost: 'Ambitions'
Number of pages: 7
129
Angels In Amplifiers: 'I'm Alright'
Number of pages: 40
785
The Apprehended: 'Still Flyin''
Number of pages: 9
167
Arcadian Alone: 'The Deathless One'
Number of pages: 2
29
Audio-Technica Demo: 'Loud And Clear'
Number of pages: 2
38
Balkun Brothers: 'So Hi. So Lo.'
Number of pages: 2
22
Bill Chudziak: 'Children Of No-one'
Number of pages: 11
218
The Black Crown: 'Cage'
Number of pages: 7
132
The Black Crown: 'Flames'
Number of pages: 12
240
Boss Fight: 'Droid'
No page information found.
168
The Brew: 'What I Want'
Number of pages: 16
311
The Bright Star Alliance: 'Error 404'
Number of pages: 3
53
The Bright Star Alliance: 'Make Like A Tree'
No page information found.
6
Candlebox: 'Happy Pills'
Number of pages: 7
124
Candlebox: 'Surrendering'
Number of pages: 4
67
Cnoc An Tursa: 'Bannockburn'
Number of pages: 7
125
The Complainiacs: 'Etc'
Number of pages: 5
88
Cunegonda: Gue

In [95]:
print(len(song_dict))

155


In [96]:
#write to a json file which will be later used to download audio files
import json
with open('6(rock_metal)_data.json', 'w') as f:
    json.dump(song_dict, f, indent=4)
    f.close()
    